In [3]:
#import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
import sys
import os

import re
import string
import nltk
import spacy
import math
from tqdm import tqdm

from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
#pd.set_option('display.max_colwidth',200)

#load spaCy model
import en_core_web_sm
nlp = en_core_web_sm.load()

matcher = Matcher(nlp.vocab)

all_files = os.listdir("HCL ML Challenge Dataset/")
print(all_files)

['X8XUG1JD.txt', 'X8XX000W.txt', 'X8XX00J6.txt', 'X8XX00LL.txt', 'X8XX00MX.txt', 'X8XX00YW.txt', 'X8XX0137.txt', 'X8XX0196.txt', 'X8XX01EH.txt', 'X8XX01EP.txt', 'X8XX01IX.txt', 'X8XX01SZ.txt', 'X8XX01TN.txt', 'X8XX01TV.txt', 'X8XX02AX.txt', 'X8XX02N7.txt', 'X8XX02SY.txt', 'X8XX036G.txt', 'X8XX03IJ.txt', 'X8XX03S9.txt', 'X8XX07H7.txt', 'X8XX0ANN.txt', 'X8XX0BU1.txt', 'X8XX0C57.txt', 'X8XX0CEA.txt', 'X8XZI954.txt', 'X8XZI9MH.txt', 'X8XZI9Y0.txt', 'X8XZIB3U.txt', 'X8XZIBDK.txt', 'X8XZIBOZ.txt', 'X8XZIBXE.txt', 'X8XZIBZD.txt', 'X8XZICIB.txt', 'X8XZICNE.txt', 'X8XZICQX.txt', 'X8XZIDG2.txt', 'X8XZIDMH.txt', 'X8XZIE17.txt', 'X8XZIEI1.txt', 'X8XZIF16.txt', 'X8XZIFE8.txt', 'X8XZIFNV.txt', 'X8XZIFZL.txt', 'X8XZIG2X.txt', 'X8XZIGW8.txt', 'X8XZIIRN.txt', 'X8XZIIW2.txt', 'X8XZIIYQ.txt', 'X8XZIJC8.txt', 'X8XZIKDF.txt', 'X8XZIKJ6.txt', 'X8XZIKOX.txt', 'X8XZIMDT.txt', 'X8XZIMKG.txt', 'X8XZIML4.txt', 'X8XZIMQ3.txt', 'X8XZIN35.txt', 'X8XZINN6.txt', 'X8XZINW9.txt', 'X8XZIO5G.txt', 'X8XZIOFU.txt', 'X8XZIO

In [4]:
def read_text_file(file):
    with open(file,'r') as fd:
        content = fd.read()
    return content

# read_text_file('HCL ML Challenge Dataset/X8XUG1JD.txt').replace('\n','')#.split('\t')

from nltk import tokenize
x = tokenize.sent_tokenize(read_text_file('HCL ML Challenge Dataset/X8XUG1JD.txt').replace('\n',''))

tokenize.sent_tokenize(read_text_file('HCL ML Challenge Dataset/X8Y4VTL4.txt').replace('\n',''))[0]

'                                                        Micro-entity Balance Sheet as at 28 February 2019                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Notes            2019            2018                                                                                                                                                                                                                                           

In [5]:
all_text = []

for i in all_files:
    all_text.append(tokenize.sent_tokenize(read_text_file('HCL ML Challenge Dataset/' + str(i)).replace('\n',''))[0])

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(all_text)
(tfidf * tfidf.T).A

array([[1.        , 0.02591751, 0.42203931, ..., 0.11193184, 0.31139507,
        0.11570282],
       [0.02591751, 1.        , 0.02945571, ..., 0.02731569, 0.02170246,
        0.02827918],
       [0.42203931, 0.02945571, 1.        , ..., 0.16582329, 0.38077109,
        0.15167629],
       ...,
       [0.11193184, 0.02731569, 0.16582329, ..., 1.        , 0.11723755,
        0.1145312 ],
       [0.31139507, 0.02170246, 0.38077109, ..., 0.11723755, 1.        ,
        0.09916694],
       [0.11570282, 0.02827918, 0.15167629, ..., 0.1145312 , 0.09916694,
        1.        ]])

In [7]:
print(tfidf[398])

  (0, 915)	0.6638997883704731
  (0, 1033)	0.2737254517025653
  (0, 294)	0.2242163177230246
  (0, 1013)	0.2821769791892773
  (0, 977)	0.16807003758078512
  (0, 889)	0.16773488271448664
  (0, 188)	0.2545884259864692
  (0, 189)	0.3470532001037285
  (0, 890)	0.16673345748967613
  (0, 885)	0.16740040288362246
  (0, 1028)	0.1687423833569875
  (0, 896)	0.1687423833569875


In [8]:
(tfidf * tfidf.T).A.shape

(500, 500)

In [9]:
tfidf.shape

(500, 1052)

In [10]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
#np.random.seed(42)

def word_tokenizer(text):
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
    return tokens

def cluster_sentences(sentences, no_of_clusters = 5):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                       stop_words=stopwords.words('english'),
                                       max_df=0.9,
                                       min_df=0.1,
                                       lowercase=True)
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    kmeans = KMeans(n_clusters = no_of_clusters)
    kmeans.fit(tfidf_matrix)
    clusters = collections.defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
        clusters[label].append(i)
    return dict(clusters)

In [11]:
np.random.seed(42)
nclusters = 15
clusters = cluster_sentences(all_text, nclusters)

C:\Users\a\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [12]:
for cluster in range(nclusters):
    print("cluster ",cluster,":")                    
    for i,sentence in enumerate(clusters[cluster]):
        print("\tsentence ",i,": ",all_text[sentence])
        print()
        print()

cluster  0 :
	sentence  0 :                                                                                 STATEMENT OF FINANCIAL POSITION                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   AS AT 30 SEPTEMBER 2019                                                                                                                                                                                                                                                   

	sentence  23 :                                                                Micro-entity Balance Sheet as at 31 May 2019                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Notes        2019                                                                                                                                                                                                                       

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
len(clusters[1])

74

In [14]:
clusters[1]

[6,
 18,
 19,
 22,
 33,
 34,
 38,
 49,
 51,
 63,
 67,
 83,
 84,
 87,
 91,
 104,
 125,
 129,
 138,
 143,
 145,
 147,
 152,
 154,
 157,
 167,
 203,
 204,
 216,
 218,
 219,
 224,
 235,
 239,
 270,
 274,
 275,
 281,
 286,
 289,
 292,
 303,
 307,
 323,
 325,
 330,
 340,
 341,
 347,
 348,
 350,
 351,
 353,
 355,
 360,
 363,
 365,
 366,
 368,
 379,
 386,
 404,
 414,
 420,
 424,
 438,
 444,
 451,
 452,
 467,
 478,
 492,
 493,
 494]

In [15]:
all_files[96]

'X8Y28AVS.txt'

In [16]:
file_cluster_2 = []
for i in clusters[2]:
    file_cluster_2.append(all_files[i].replace('.txt',''))
print(file_cluster_2)

['X8XZIMQ3', 'X8Y28AVS', 'X8Y4QO1V', 'X8Y4QVUG', 'X8Y4SHTK', 'X8Y4SLB7', 'X8Y4SNUY', 'X8Y4SP28', 'X8Y4T2MJ', 'X8Y4T6GG', 'X8Y4T78O', 'X8Y4T8P4', 'X8Y4TDOJ', 'X8Y4U0G3', 'X8Y4U4ZM', 'X8Y4U77U', 'X8Y4U8V6', 'X8Y4UBJF', 'X8Y4UKFK', 'X8Y4UVSY']


In [17]:
file_cluster_1 = []
for i in clusters[1]:
    file_cluster_1.append(all_files[i].replace('.txt',''))
print(file_cluster_1)

['X8XX0137', 'X8XX03IJ', 'X8XX03S9', 'X8XX0BU1', 'X8XZICIB', 'X8XZICNE', 'X8XZIE17', 'X8XZIJC8', 'X8XZIKJ6', 'X8XZIOV4', 'X8XZIPGP', 'X8XZIVMX', 'X8XZIW68', 'X8Y26BQO', 'X8Y287W0', 'X8Y28E4O', 'X8Y28LXS', 'X8Y28MNT', 'X8Y2ER1U', 'X8Y2ETP7', 'X8Y2EY7M', 'X8Y2EYNK', 'X8Y2F0L6', 'X8Y2F337', 'X8Y2F3V7', 'X8Y4QO6V', 'X8Y4R7EY', 'X8Y4R7MZ', 'X8Y4RG7U', 'X8Y4RH9C', 'X8Y4RHX7', 'X8Y4RLJL', 'X8Y4RREB', 'X8Y4RUDE', 'X8Y4S969', 'X8Y4SDBE', 'X8Y4SE57', 'X8Y4SFR7', 'X8Y4SKV5', 'X8Y4SMXM', 'X8Y4SOZS', 'X8Y4SWQ1', 'X8Y4SYH4', 'X8Y4T8PC', 'X8Y4TAXD', 'X8Y4TD2I', 'X8Y4TKGH', 'X8Y4TKLK', 'X8Y4TU43', 'X8Y4TUMQ', 'X8Y4TUUB', 'X8Y4TV08', 'X8Y4TVGA', 'X8Y4TW66', 'X8Y4TZVK', 'X8Y4U0TL', 'X8Y4U2BD', 'X8Y4U37L', 'X8Y4U3UP', 'X8Y4U8YA', 'X8Y4UC77', 'X8Y4UKE8', 'X8Y4UNWI', 'X8Y4URYJ', 'X8Y4UT6H', 'X8Y4V4PZ', 'X8Y4V99N', 'X8Y4VG20', 'X8Y4VGOP', 'X8Y4VPEP', 'X8Y4W2YW', 'X8Y4WFEP', 'X8Y4WFMO', 'X8Y4WHBC']


In [16]:
#doc = nlp(all_text[13])

# for tok in doc:
#   print(tok.text, "--->", tok.dep_, '--->', tok.pos_)

In [18]:
matcher = Matcher(nlp.vocab)
file_cluster_2_values = []
#small = []
def current_asset_pattern_2(sent):
    #file_cluster_2_values = []
#     np.random.seed(42)
    
    pattern1 = [{'ORTH':'Current'},
                {'LOWER':'assets'}]
    
    pattern2 = [{'POS':'NOUN'},
                {'LOWER':'at'},
                {'LOWER':'bank'},
                {'LOWER':'and'},
                {'LOWER':'in'},
                {'LOWER':'hand'}]
    
    pattern3 = [{'ORTH':'Net'},
                {'LOWER':'assets'}]
    
    pattern4 = [{'ORTH':'Issued'},
                {'LOWER':'share'},
                {'LOWER':'capital'}]
    
    pattern5 = [{'ORTH':'1'},
                {'ORTH':'Ordinary'},
                {'ORTH':'Share'},
                {'ORTH':'1'},
                {'ORTH':'of'},
                {'ORTH':'Â£'},
                {'LOWER':'each'}]
    
    pattern6 = [{'ORTH':'Total'},
                {'ORTH':'Shareholder'},
                {'LOWER':'funds'}]
    
    matcher.add('rule1',None,pattern1)
    matcher.add('rule2',None,pattern2)
    matcher.add('rule3',None,pattern3)
    matcher.add('rule4',None,pattern4)
    matcher.add('rule5',None,pattern5)
    matcher.add('rule6',None,pattern6)
    
    matches = matcher(nlp(sent))
    
#     span = doc[matches[0][1]:matches[0][2]]
#     print(span.text)

    small = []
    for match_id,start,end in matches:
        #print(start,end)
        span = nlp(sent)[start:end]
        small.append(span.text)
        print(span.text)
    file_cluster_2_values.append(small)
    print()

In [19]:
matcher = Matcher(nlp.vocab)
file_cluster_1_values = []
#small = []
def current_asset_pattern_1(sent):
    #file_cluster_2_values = []
#     np.random.seed(42)
    
    pattern1 = [{'ORTH':'Net'},
                {'LOWER':'assets'}]
    
    pattern2 = [{'ORTH':'Called'},
                {'LOWER':'up'},
                {'LOWER':'share'},
                {'LOWER':'capital'},
                {'LOWER':'not'},
                {'LOWER':'paid'}]
    
    pattern3 = [{'ORTH':'Issued'},
                {'LOWER':'share'},
                {'LOWER':'capital'}]
    
    pattern4 = [{'ORTH':'100'},
                {'ORTH':'Ordinary'},
                {'ORTH':'Shares'},
                {'ORTH':'1'},
                {'ORTH':'of'},
                {'ORTH':'Â£'},
                {'LOWER':'each'}]
    
    pattern5 = [{'ORTH':'Total'},
                {'ORTH':'Shareholder'},
                {'LOWER':'funds'}]
    
    pattern6 = [{'ORTH':'Current'},
                {'LOWER':'assets'}]
    
    matcher.add('rule1',None,pattern1)
    matcher.add('rule2',None,pattern2)
    matcher.add('rule3',None,pattern3)
    matcher.add('rule4',None,pattern4)
    matcher.add('rule5',None,pattern5)
    matcher.add('rule6',None,pattern6)
    
    matches = matcher(nlp(sent))
    
#     span = doc[matches[0][1]:matches[0][2]]
#     print(span.text)

    small = []
    for match_id,start,end in matches:
        #print(start,end)
        span = nlp(sent)[start:end]
        small.append(span.text)
        print(span.text)
    file_cluster_1_values.append(small)
    print()

In [20]:
#file_cluster_2_values = []
for i in clusters[2]:
    current_asset_pattern_2(all_text[i])

Net assets
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net assets
Issued share capital
1 Ordinary Share 1 of Â£ each
Total Shareholder funds

Current assets
Cash at bank and in hand
Net a

In [21]:
#file_cluster_2_values = []
for i in clusters[1]:
    current_asset_pattern_1(all_text[i])

Called up share capital not paid
Net assets
Net assets
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Called up share capital not paid
Net assets
Net assets
Total Shareholder funds
Total Shareholder funds

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
Total Shareholder funds
Total Shareholder funds

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
Total Shareholder funds
Total Shareholder funds

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Called up share capit

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Called up share capital not paid
Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Called up share capital not paid

Called up share capital not paid
Net assets
Net assets
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Called up share capital not paid
Net assets
Net assets
100 Ordinary Shares 1 of Â£ each
Total Shareholder funds
Total Shareholder funds

Current assets
Current assets
Cash at bank and in hand
Net assets
Net assets
Issued share capital
Issued share capital
100 Ordinary Share

In [21]:
file_cluster_2_values

[['Net assets', '1 Ordinary Share 1 of Â£ each', 'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Shareholder funds'],
 ['Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Issued share capital',
  '1 Ordinary Share 1 of Â£ each',
  'Total Sh

In [22]:
file_cluster_1_values

[['Called up share capital not paid',
  'Net assets',
  'Net assets',
  '100 Ordinary Shares 1 of Â£ each',
  'Total Shareholder funds',
  'Total Shareholder funds'],
 ['Called up share capital not paid',
  'Net assets',
  'Net assets',
  'Total Shareholder funds',
  'Total Shareholder funds'],
 ['Current assets',
  'Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Net assets',
  'Issued share capital',
  'Issued share capital',
  'Total Shareholder funds',
  'Total Shareholder funds'],
 ['Current assets',
  'Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Net assets',
  'Issued share capital',
  'Issued share capital',
  '100 Ordinary Shares 1 of Â£ each',
  'Total Shareholder funds',
  'Total Shareholder funds'],
 ['Current assets',
  'Current assets',
  'Cash at bank and in hand',
  'Net assets',
  'Net assets',
  'Issued share capital',
  'Issued share capital',
  'Total Shareholder funds',
  'Total Shareholder funds'],
 ['Current assets',
  'Curr

In [23]:
# dict_value = {}
# dict_value['Current assets'] = 'nan'
# dict_value['Cash at bank and in hand'] = '1'
# dict_value['Net assets'] = '1'
# dict_value['Issued share capital'] = 'nan'
# dict_value['1 Ordinary Share 1 of Â£ each'] = '1'
# dict_value['Total Shareholder funds'] = '1'
# dict_value

In [22]:
df = pd.DataFrame()
df['Filename'] = file_cluster_2
df['x'] = file_cluster_2_values
df['Extracted Values'] = ''

In [25]:
# df.head()

In [26]:
# df['x'][0]

In [23]:
#dic = {}
for k,i in enumerate(df['x']):
    dic = {}
    for j in i:
        #print(j)
        if j in ['Cash at bank and in hand','Net assets','1 Ordinary Share 1 of Â£ each','Total Shareholder funds']:
            dic[j] = '1'
        else:
            dic[j] = 'nan'
    #print()
    df['Extracted Values'][k] = dic

In [24]:
df1 = pd.DataFrame()
df1['Filename'] = file_cluster_1
df1['x'] = file_cluster_1_values
df1['Extracted Values'] = ''

In [25]:
#dic = {}
for k,i in enumerate(df1['x']):
    dic1 = {}
    for j in i:
        #print(j)
        if j in ['Net assets','Called up share capital not paid','Cash at bank and in hand','100 Ordinary Share 1 of Â£ each','Total Shareholder funds']:
            dic1[j] = '100'
        else:
            dic1[j] = 'nan'
    #print()
    df1['Extracted Values'][k] = dic1

In [26]:
#dic

In [27]:
#df['Extracted Values'][0]

In [28]:
df.head()

,Filename,x,Extracted Values
0,X8XZIMQ3,"[Net assets, 1 Ordinary Share 1 of Â£ each, To...","{'Net assets': '1', '1 Ordinary Share 1 of Â£ ..."
1,X8Y28AVS,"[Current assets, Cash at bank and in hand, Net...","{'Current assets': 'nan', 'Cash at bank and in..."
2,X8Y4QO1V,"[Current assets, Cash at bank and in hand, Net...","{'Current assets': 'nan', 'Cash at bank and in..."
3,X8Y4QVUG,"[Current assets, Cash at bank and in hand, Net...","{'Current assets': 'nan', 'Cash at bank and in..."
4,X8Y4SHTK,"[Current assets, Cash at bank and in hand, Net...","{'Current assets': 'nan', 'Cash at bank and in..."


In [29]:
submission_file = pd.DataFrame()
f_name = []
for i in all_files:
    f_name.append(i.replace('.txt',''))
submission_file['Filename'] = f_name
submission_file['Extracted Values'] = ''

In [30]:
submission_file['Filename'][96]

'X8Y28AVS'

In [31]:
submission_file.head()

,Filename,Extracted Values
0,X8XUG1JD,
1,X8XX000W,
2,X8XX00J6,
3,X8XX00LL,
4,X8XX00MX,


In [36]:
for k,i in enumerate(df['Filename']):
    submission_file['Extracted Values'] = np.where((submission_file['Filename'] == i),df['Extracted Values'][k],submission_file['Extracted Values'])

In [37]:
for k,i in enumerate(df1['Filename']):
    submission_file['Extracted Values'] = np.where((submission_file['Filename'] == i),df1['Extracted Values'][k],submission_file['Extracted Values'])

In [38]:
submission_file['Extracted Values'][56]

{'Net assets': '1',
 '1 Ordinary Share 1 of Â£ each': '1',
 'Total Shareholder funds': '1'}

In [39]:
submission_file['Filename'][96]

'X8Y28AVS'

In [40]:
submission_file.to_csv('hcl_ml_sub1000.csv', encoding='utf-8', index=False)

In [41]:
submission_file.dtypes

Filename            object
Extracted Values    object
dtype: object

In [42]:
submission_file['Filename'] = submission_file['Filename'].astype('str') 
# submission_file['Extracted Values'] = submission_file['Extracted Values'].astype('dict') 